In [13]:
from __future__ import division
import pandas as pd
import numpy as np
import operator
import pickle

In [2]:
df = pd.read_csv("data/nodes.tsv", sep='\t')

In [3]:
df

,ID,Name,Depth
0,HP:0000001,All,0
1,HP:0012823,Clinical modifier,1
2,HP:0000118,Phenotypic abnormality,1
3,HP:0032223,Blood group,1
4,HP:0031797,Clinical course,1
...,...,...,...
15242,HP:0009952,Complete duplication of the middle phalanx of ...,15
15243,HP:0009951,Partial duplication of the distal phalanx of t...,15
15244,HP:0009979,Complete duplication of the middle phalanx of ...,15
15245,HP:0009978,Complete duplication of the distal phalanx of ...,15


In [8]:
df_new =df.loc[(df['Depth'] == 1) | (df['Depth']==2)]

In [9]:
df_new

,ID,Name,Depth
1,HP:0012823,Clinical modifier,1
2,HP:0000118,Phenotypic abnormality,1
3,HP:0032223,Blood group,1
4,HP:0031797,Clinical course,1
5,HP:0032443,Past medical history,1
...,...,...,...
72,HP:0001608,Abnormality of the voice,2
73,HP:0032224,ABO blood group,2
74,HP:0000007,Autosomal recessive inheritance,2
75,HP:0041158,Fractured trunk,2


In [14]:
with open('graph_ontology.pkl', 'rb') as input:
    graph_load = pickle.load(input)

In [21]:
df.loc[df['ID'] == 'HP:0012824']

,ID,Name,Depth
76,HP:0012824,Severity,2


In [23]:
Phenotypic_abnor_descendants = graph_load.get_descendants('HP:0000118')
for i in Phenotypic_abnor_descendants:
    df.loc[df['ID'] == 'i']